In [18]:
val input = """32T3K 765
                T55J5 684
                KK677 28
                KTJJT 220
                QQQJA 483"""


val i = "32T3K 765"


enum class HandType {
    HighCard, OnePair, TwoPairs, ThreeOfAKind, FullHouse, FourOfAKind, FiveOfAKind
}

data class Hand(val cards: List<Char>, val bid: Int) {

    companion object {
        fun of(input: String): Hand {
            val cards = input.trim().substringBefore(" ").toCharArray().toList()
            val bid = input.trim().substringAfter(" ").toInt()
            return Hand(cards, bid)
        }
    }

    override fun toString(): String {
        return "$cards:$bid"
    }

    fun type(): HandType {
        val grouped = cards.groupingBy { it }.eachCount()
        val cardsCount = grouped.values.sorted()
        val jCounts = grouped['J'] ?: 0
        if (cardsCount == listOf(5)) {
            return HandType.FiveOfAKind
        }

        if (cardsCount == listOf(1, 4)) {
            return when (jCounts) {
                1, 4 -> HandType.FiveOfAKind
                else -> HandType.FourOfAKind
            }
        }

        if (cardsCount == listOf(2, 3)) {
            return when (jCounts) {
                2, 3 -> HandType.FiveOfAKind
                else -> HandType.FullHouse
            }
        }

        if (cardsCount == listOf(1, 1, 3)) {
            return when (jCounts) {
                1 -> HandType.FourOfAKind
                else -> HandType.ThreeOfAKind
            }
        }

        if (cardsCount == listOf(1, 2, 2)) {
            return when (jCounts) {
                1 -> HandType.FullHouse
                2 -> HandType.FourOfAKind
                else -> HandType.TwoPairs
            }
        }

        if (cardsCount == listOf(1, 1, 1, 2)) {
            return when (jCounts) {
                1, 2 -> HandType.ThreeOfAKind
                else -> HandType.OnePair
            }


        }
        return when (jCounts) {
            1 -> HandType.OnePair
            else -> HandType.HighCard
        }
    }

}

val hands = input.split("\n").map { Hand.of(it) }

val cardRankOrder = mapOf(
    '2' to 2,
    '3' to 3,
    '4' to 4,
    '5' to 5,
    '6' to 6,
    '7' to 7,
    '8' to 8,
    '9' to 9,
    'T' to 10,
    'J' to 1,
    'Q' to 12,
    'K' to 13,
    'A' to 14,
)

fun makeCombinedComparator(order: Map<Char, Int>) = compareBy<Hand> { it.type() }.thenComparator { h1, h2 ->
    h1.cards.zip(h2.cards)
        .mapNotNull { (c1, c2) ->
            val order1 = order[c1] ?: 0
            val order2 = order[c2] ?: 0
            if (order1 != order2) order1.compareTo(order2) else null
        }
        .firstOrNull() ?: 0
}


val combinedComparator = makeCombinedComparator(cardRankOrder)

val r = hands.sortedWith(combinedComparator).map { it -> Pair(it, it.type()) }
    .mapIndexed { idx, (hand, _) -> (idx + 1) * hand.bid }.sum()


//, {it.cards.joinToString("")}
println(r)

5905
